In [5]:
'''
  All impoorts modules n libs stay here in this block
'''

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
import pandas as pd
import numpy as np
from tqdm import tqdm  # To watch training via loadin bars
import matplotlib.pyplot as plt

# Custom Modules
from src.dataset import SatelliteDataset
from src.models import ValuationModel

# Hyperparameters
BATCH_SIZE = 32        # Study 32 houses at a time
LEARNING_RATE = 0.001  # How fast the AI updates its beliefs/ the alpha looking parameter in eqn.
NUM_EPOCHS = 15        # TO make it read the entire dataset 15 times
SEED = 42              # Ensures we get the same result every time
                       # Debuggin and proof purposes.

#### Run the cell below to verify whether training is happening on GPU or CPU.

In [6]:
# Setup Device (Uses GPU if you have one, otherwise CPU)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device For Training: {DEVICE}")

# Set Random Seed for Consistency
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

Device For Training: cuda


#### Now we Load the DATA

In [ ]:
print("Loading cleaned Data")

# Loading the CSVs created during preprocessing.ipynb
# We use the cleaned versions because they have 0s instead of NaNs
train_df = pd.read_csv("data/train_cleaned.csv")
test_df = pd.read_csv("data/test_cleaned.csv")

print(f"Data Loaded")
print(f"   - Train Rows: {len(train_df)}")
print(f"   - Test Rows:  {len(test_df)}")

# Defining the Feature Columns/
# Its like telling the Dataset which columns are "inputs"
feature_cols = [
    "bedrooms",
    "bathrooms",
    "sqft_living",
    "sqft_lot",
    "floors",
    "waterfront",
    "view",
    "condition",
    "grade",
    "sqft_above",
    "sqft_basement",
    "yr_built",
    "yr_renovated",
    "sqft_living15",
    "sqft_lot15",
    "lat",
    "long",
]

# A quick Check Ensure we have 17 columns
print(f"   - Features used: {len(feature_cols)}")